# EM4QS Demo 22.01.2026


## Library installation
We have not yet publicised the library in Pypi, so for now you can build it from source. 

```
git clone https://github.com/QuantumHel/syn.git
cd syn
cargo build
pip install .
```

In [1]:
from qiskit.quantum_info import Clifford, Operator
from qiskit import QuantumCircuit


from synpy.qiskit.plugin import SynPyCliffordPlugin, qiskit_to_synir
from synpy.synpy_rust import QiskitSynIR

circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0, 1)
circuit.rz(1.5, 1)
sample_circuit = circuit.copy()

print(circuit)

     ┌───┐                
q_0: ┤ H ├──■─────────────
     └───┘┌─┴─┐┌─────────┐
q_1: ─────┤ X ├┤ Rz(1.5) ├
          └───┘└─────────┘
q_2: ─────────────────────
                          


In [2]:
def our_transpile(circuit: QuantumCircuit) -> QuantumCircuit:
    synir_result = QiskitSynIR(circuit.copy_empty_like())
    pe_wrap = qiskit_to_synir(circuit)
    pe_wrap.synthesize_to_qiskit(synir_result)
    return synir_result.get_circuit()

In [3]:
new_circuit = our_transpile(circuit)
print(new_circuit)

     ┌───┐                
q_0: ┤ H ├──■─────────────
     └───┘┌─┴─┐┌─────────┐
q_1: ─────┤ X ├┤ Rz(1.5) ├
          └───┘└─────────┘
q_2: ─────────────────────
                          


In [10]:
def check_equivalent(circuit1: QuantumCircuit, circuit2: QuantumCircuit) -> bool:
    op1 = Operator.from_circuit(circuit1)
    op2 = Operator.from_circuit(circuit2)
    return op1.equiv(op2)

In [11]:
check_equivalent(new_circuit, sample_circuit)

True

# Random circuits

In [15]:
from qiskit.circuit.library import QuantumVolume

qv = QuantumVolume(5).decompose().decompose()
qv.draw(fold=200)
qv.count_ops()

OrderedDict([('u', 80), ('cx', 30)])

In [17]:
qiskit_qv = transpile(qv, optimization_level=3)
qiskit_qv.count_ops()

OrderedDict([('u', 41), ('cx', 24), ('u3', 11), ('unitary', 1)])

In [18]:
syn_qv = our_transpile(qv)
syn_qv.count_ops()

OrderedDict([('rz', 111), ('h', 100), ('cx', 37)])

In [20]:
check_equivalent(qiskit_qv, qv)

True

In [21]:
check_equivalent(qv, syn_qv)

False

# Compiling Shor's 21
The [quantum program to factor 21](https://algassert.com/post/2500) is much too long to execute on current hardware. But we can improve the cost with our library.

In [5]:
from qiskit import qasm2

original_shor = qasm2.load("factor21.qasm")
print(original_shor.count_ops())
original_shor.draw(fold=200)

OrderedDict({'ccx': 369, 'cx': 191, 'x': 73, 'u': 45, 'h': 20, 'measure': 10})


/var/folders/3q/vjbn2sl16r1g41lg5pk752khfzf5fm/T/ipykernel_39318/1544880673.py:3: FutureWarning: Conditioned gates in qasm2 will be loaded as an IfElseOp starting in Qiskit 2.0
  original_shor = qasm2.load("factor21.qasm")


┌───┐┌───┐     ┌───┐                         ┌───┐          ┌───┐                                             ┌───┐                                   ┌───┐     ┌───┐          ┌───┐     »
    val0: ┤ X ├┤ X ├─────┤ X ├─────────────────────────┤ X ├──■───────┤ X ├──■────────────────────────■─────────────────┤ X ├──────────────────────■────────────┤ X ├──■──┤ X ├───────■──┤ X ├─────»
          └───┘└─┬─┘┌───┐└─┬─┘                         └─┬─┘  │       └─┬─┘  │            ┌───┐       │                 └─┬─┘┌───┐                 │            └─┬─┘  │  └─┬─┘       │  └─┬─┘     »
    val1: ───────┼──┤ X ├──┼─────────────────────────────┼────┼─────────┼────┼────────────┤ X ├───────┼─────────■─────────┼──┤ X ├───────■─────────┼─────────■────┼────┼────■─────────┼────┼───────»
          ┌───┐  │  └─┬─┘  │  ┌───┐               ┌───┐  │  ┌─┴─┐       │    │            └─┬─┘       │  ┌───┐  │  ┌───┐  │  └─┬─┘     ┌─┴─┐       │         │    │  ┌─┴─┐  │         │    │       »
    val2: ┤ X ├──┼────┼────┼──┤ X ├───────■───────┤ X ├──■──┤ X ├───────■────■──────────────┼─────────■──┤ X ├──┼──┤ X ├──┼────┼────■──┤ X ├──■────┼────■────┼────┼──┤ X ├──┼─────────■────┼───────»
          └─┬─┘  │    │    │  └─┬─┘       │       └─┬─┘  │  └─┬─┘┌───┐  │    │  ┌───┐       │         │  └─┬─┘┌─┴─┐└───┘  │    │    │  └─┬─┘┌─┴─┐  │  ┌─┴─┐┌─┴─┐  │  └─┬─┘  │         │    │       »
    val3: ──■────┼────┼────┼────┼────■────┼────■────┼────■────┼──┤ X ├──■────┼──┤ X ├───────■─────────┼────┼──┤ X ├──■────■────■────■────┼──┤ X ├──■──┤ X ├┤ X ├──■────■────┼────■────┼────■────■──»
                 │    │    │    │    │  ┌─┴─┐  │    │         │  └─┬─┘     ┌─┴─┐└─┬─┘┌───┐  │  ┌───┐┌─┴─┐  │  └─┬─┘┌─┴─┐  │    │  ┌─┴─┐  │  └───┘┌─┴─┐└───┘└─┬─┘  │         │  ┌─┴─┐┌─┴─┐       │  »
    val4: ───────┼────┼────┼────■────┼──┤ X ├──┼────■─────────┼────┼───────┤ X ├──┼──┤ X ├──■──┤ X ├┤ X ├──┼────┼──┤ X ├──┼────■──┤ X ├──┼───────┤ X ├───────┼────┼─────────┼──┤ X ├┤ X ├───────┼──»
          ┌───┐  │    │    │    │    │  └─┬─┘  │    │         │    │       └───┘  │  └───┘     └───┘└───┘  │    │  └───┘  │       └───┘  │       └───┘       │    │         │  └───┘└───┘       │  »
    exp0: ┤ H ├──■────■────┼────┼────┼────┼────┼────┼─────────┼────┼──────────────┼────────────────────────┼────┼─────────┼──────────────┼───────────────────┼────┼─────────┼───────────────────┼──»
          ├───┤            │    │  ┌─┴─┐  │  ┌─┴─┐  │         │    │              │                        │    │         │              │                   │    │         │                 ┌─┴─┐»
    exp1: ┤ H ├────────────■────■──┤ X ├──■──┤ X ├──■─────────■────■──────────────■────────────────────────■────■─────────■──────────────■───────────────────■────■─────────■─────────────────┤ X ├»
          ├───┤                    └───┘     └───┘                                                                                                                                            └───┘»
    exp2: ┤ H ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
          ├───┤                                                                                                                                                                                    »
    exp3: ┤ H ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
          ├───┤                                                                                                                                                                                    »
    exp4: ┤ H ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────»
          ├───┤                                                       

This circuit contains gates that are not yet natively supported by our library. 
The QFT is implemented as a fourier basis umeasurement, which we can temporarily remove and place back.

In [6]:
no_qft = original_shor.copy_empty_like()
for gate in original_shor.data: 
    if gate.name not in ["measure", "u"]:
        no_qft.compose(gate, gate.qubits, gate.clbits, inplace=True)
no_qft.count_ops()

OrderedDict([('ccx', 369), ('cx', 191), ('x', 73), ('h', 20)])

In [7]:
from qiskit import transpile

basic_shor = transpile(no_qft, basis_gates=["cx", "h", "x", "rz"], optimization_level=0)
basic_shor.count_ops()

OrderedDict([('rz', 2583), ('cx', 2405), ('h', 758), ('x', 73)])

In [8]:
qiskit_shor = transpile(no_qft, optimization_level=3)
qiskit_shor.count_ops()

OrderedDict([('cx', 2373),
             ('t', 1151),
             ('tdg', 1091),
             ('u2', 326),
             ('h', 320),
             ('x', 47),
             ('unitary', 8)])

In [9]:
syn_shor = our_transpile(no_qft)
syn_shor.count_ops()

OrderedDict([('cx', 3840),
             ('rz', 2605),
             ('h', 1306),
             ('sx', 412),
             ('s', 9),
             ('x', 7),
             ('z', 6)])

In [ ]:
check_equivalent(no_qft, syn_shor)

In [ ]:
check_equivalent(no_qft, qiskit_shor)

In [12]:
check_equivalent(no_qft, basic_shor)

KeyboardInterrupt: 